In [2]:
from collections import namedtuple
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Natasha

# Подготавливаем данные 

In [3]:
# Предполагаем, что у каждого класса есть свой класс и в каждом классе проставляем номер курса класса

CLASS_TABLE = np.zeros(5 * 8)

CLASSROOMS = ['r1','r2','r3','r4','r5','r6','r7','r8']
CLASSES = {'5а':['Математика',     5, # 2
                 'Русский',        6, # 2
                 'Литература',     3, # 1.5
                 'ИЗО',            1, # 1
                 'Музыка',         1, # 1
                 'Физкультура',    2, # 1
                 'Труд',           2, # 1
                 'История',        2, # 1.5
                 'География',      2, # 1.5
                 'Информатика',    1, # 1.5
                 'Немецкий язык',  2, # 1.5
                 'Биология',       1  # 1.5 
                ], 
           '5б':['Математика',     5,
                 'Русский',        6,
                 'Литература',     3,
                 'ИЗО',            1,
                 'Музыка',         1,
                 'Физкультура',    2,
                 'Труд',           2,
                 'История',        2,
                 'География',      2,
                 'Информатика',    1,
                 'Немецкий язык',  2,
                 'Биология',       1
                ],
           '6а':['Математика',      5, # 2
                 'Русский',         7, # 2
                 'Литература',      3, # 1.5
                 'ИЗО',             1, # 1
                 'Музыка',          1, # 1
                 'Физкультура',     3, # 1
                 'Труд',            2, # 1
                 'История',         2, # 1.5
                 'География',       1, # 1.5
                 'Информатика',     1, # 1.5
                 'Немецкий язык',   3, # 1.5
                 'Французский язык',2, # 1.5
                 'Общестовазнание', 1, # 1.5
                 'Биология',        1  # 1.5
                ], 
           '6б':['Математика',       5,
                 'Русский',          7,
                 'Литература',       3,
                 'ИЗО',              1,
                 'Музыка',           1,
                 'Физкультура',      3,
                 'Труд',             2,
                 'История',          2,
                 'География',        1,
                 'Информатика',      1,
                 'Немецкий язык',    3,
                 'Французский язык', 2,
                 'Общестовазнание',  1,
                 'Биология',         1
                ],
           '7а':['Алгебра',          3, # 2
                 'Геометрия',        2, # 2
                 'Русский',          5, # 2
                 'Литература',       2, # 1.5
                 'ИЗО',              1, # 1
                 'Музыка',           1, # 1
                 'Физкультура',      3, # 1
                 'Труд',             2, # 1
                 'История',          3, # 2
                 'География',        2, # 2
                 'Информатика',      1, # 2
                 'Немецкий язык',    2, # 2
                 'Французский язык', 2, # 2
                 'Общестовазнание',  1, # 2
                 'Биология',         1, # 2
                 'Физика',           2, # 2
                 'ОБЖ',              1  # 1
                ],
           '7б':['Алгебра',          3,
                 'Геометрия',        2,
                 'Русский',          5,
                 'Литература',       2,
                 'ИЗО',              1,
                 'Музыка',           1,
                 'Физкультура',      3,
                 'Труд',             2,
                 'История',          3,
                 'География',        2,
                 'Информатика',      1,
                 'Немецкий язык',    2,
                 'Французский язык', 2,
                 'Общестовазнание',  1,
                 'Биология',         1,
                 'Физика',           2, # 2
                 'ОБЖ',              1
                ],
           '8а':['Алгебра',          4,
                 'Геометрия',        2,
                 'Русский',          5,
                 'Литература',       2,
                 'ИЗО',              1,
                 'Музыка',           1,
                 'Физкультура',      3,
                 'Труд',             1,
                 'История',          2,
                 'География',        2,
                 'Информатика',      1,
                 'Немецкий язык',    3,
                 'Общестовазнание',  1,
                 'Биология',         2,
                 'Физика',           2,
                 'ОБЖ',              1,
                 'Химия',            2 # 2
                ],
           '9а':['Алгебра',         4,
                 'Геометрия',       2,
                 'Русский',         5,
                 'Литература',      3,
                 #'ИЗО',            1
                 #'Музыка',       # 1
                 'Физкультура',     3,
                 #'Труд',         # 1
                 'История',         3,
                 'География',       2,
                 'Информатика',     1,
                 'Немецкий язык',   3,
                 'Французский язык', 1,
                 'Общестовазнание',  1,
                 'Биология',        2,
                 'Физика',          3,
                 'ОБЖ',             1,
                 'Химия',           2
                ]          
          }

TEACHERS = ['Чуркина Наталья Аркадьевна',      # Немецкий язык __ французский
            'Просвирин Сергей Дмитриевич',     # История __ Технология
            'Митрохина Альбина Николаевна',    # Математика
            'Захарова Ирина Ивановна',         # Математика __ Физика
            'Суханова Анна Львовна',           # Биологии __ Химии
            'Ершова Светлана Николаевна',      # Физра __ География
            'Селезнева Светлана Валерьевна',   # История Информатика
            'Самарина Валентина Ивановна',     # Немецкий язык __ Французский
            'Сухарева Оксана Николаевна',      # Русский язык __ Литература
            'Меликаева Налия Кирилловна',      # Изо
            'Сидорова Софья Александровна',    # Русский язык __ Литература __ География
            'Соловьева Розалия Андреевна',     # Русский язык __ Литература
            'Тукова Елена Николаевна',         # Технология __ ОБЖ
            'Похвалий Екатерина Михайловна',   # Музыка
            'Сорокина Анастасия Александровна' # Русский язык __ Литература
           ]

lst = ['ItemName','TeacherFIO','Class','count_lessons']
courseDescription = namedtuple('courseDescription', lst)

COURSES_5a = {}
COURSES_5б = {}
COURSES_6a = {}
COURSES_6б = {}
COURSES_7а = {}
COURSES_7б = {}
COURSES_8а = {}
COURSES_9а = {}


for class_, items in CLASSES.items():
    KEY_COURSE = 1
    for count, item in enumerate( zip(items[0::2], items[1::2]) ):
        lesson = item[0]
        count_lessons_per_week = item[1]
                                 
        if class_ in ['5а','5б','6а','6б','7а'] and lesson in ['Французский язык','Немецкий язык']:
            teacher = TEACHERS[7]
        elif lesson in ['Французский язык','Немецкий язык']:
            teacher = TEACHERS[0]
        
        if class_ in ['5а','5б','6а','6б'] and lesson == 'История':
            teacher = TEACHERS[1]
        elif lesson == 'История':
            teacher = TEACHERS[6]
        
        if class_ in ['5а','5б','6а'] and lesson == 'Труд':
            teacher = TEACHERS[12]
        elif lesson == 'Труд':
            teacher = TEACHERS[1]                         
        
        if (class_ in ['7а','8а','9а'] and lesson in ['Алгебра','Геометрия']) or (lesson == 'Физика'):
            teacher = TEACHERS[3]
        elif lesson in ['Алгебра','Геометрия','Математика']:
            teacher = TEACHERS[2]
            
        if lesson in ['Биология','Химия']:
            teacher = TEACHERS[4]
                                 
        if lesson == 'ИЗО':
            teacher = TEACHERS[9]
                            
        if lesson == 'Музыка':
            teacher = TEACHERS[-2]
        
        if lesson == 'ОБЖ':
            teacher = TEACHERS[-3]
        
        if lesson == 'Информатика':
            teacher = TEACHERS[6]
                                 
        if (lesson == 'Физкультура') or (lesson == 'География' and class_ in ['5а','5б']):
            teacher = TEACHERS[5]
        elif (lesson == 'География') or (lesson in ['Русский', 'Литература'] and class_  == '5а'):
            teacher = TEACHERS[10]
                                 
        if lesson in ['Русский', 'Литература'] and class_  in ['5б','6а']:
            teacher = TEACHERS[8]
        
        if lesson in ['Русский', 'Литература'] and class_  in ['6б','9а']:
            teacher = TEACHERS[-4]
        
        if lesson in ['Русский', 'Литература'] and class_  in ['7а','7б','8а']:
            teacher = TEACHERS[-1]
                                 
        f = courseDescription(item, teacher, class_, count_lessons_per_week)  
        
        # Создаем словари для СЛОВАРЯ
        if class_ == '5а':
#            COURSES_5a.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 20) })
            COURSES_5a.update({KEY_COURSE :  f})
            
        
        if class_ == '5б':
#            COURSES_5б.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 20) })
            COURSES_5б.update({KEY_COURSE :  f})
            
            
        if class_ == '6а':
#            COURSES_6a.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 15) })
            COURSES_6a.update({KEY_COURSE :  f})
            
            
        if class_ == '6б':
#            COURSES_6б.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 15) })
            COURSES_6б.update({KEY_COURSE :  f})
            
        
        if class_ == '7а':
#            COURSES_7а.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 14) })
            COURSES_7а.update({KEY_COURSE :  f})
            
        if class_ == '7б':
#            COURSES_7б.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 14) })
            COURSES_7б.update({KEY_COURSE :  f})
            
        if class_ == '8а':
#            COURSES_8а.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 13) })
            COURSES_8а.update({KEY_COURSE :  f})
            
        if class_ == '9а':
#            COURSES_9а.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 12) })
            COURSES_9а.update({KEY_COURSE :  f})
            

        KEY_COURSE += 1
#COURSES.update({0: courseDescription('Ничего', 'Ничего', 'Ничего', 0) })
# len(COURSES)
COURSES = [COURSES_5a,
           COURSES_5б,
           COURSES_6a,
           COURSES_6б,
           COURSES_7а,
           COURSES_7б,
           COURSES_8а,
           COURSES_9а]

# Создание эталлонного расписания 

Поэтапное описание алгоритма:
1) Определяем изначальные параметры
    1.1 Порог на кол-во уроков в одном дне. 8 Стандартный порог.
    1.2 Параметр "стоимость предметов" (BASIC_LVL_LESSONS) . Определяется статическим словарем, где сложный предмет - 2 очка, средний - 1.5, легкий - 1.
    1.3 Параметр "кол-во уроков у класса". 
        1.3.1 На первом этапе определяем сколько дней в недели может учится класс - 5 или 6. По стандарту определяем, что 9 класс учится 6 дней. 
        1.3.2 общее кол-во уроков класса в неделю делим на кол-во дней из п.1.3.1 и округляем вниз.
        1.3.3 Число из п.1.3.2 умножаем на число из п.1.3.1 и вычитаем из него общее кол-во уроков. Получаем кол-во уроков, которое необходимо добавить к п.1.3.2, но так, чтобы порог по кол-ву уроков в одном дне не превышал порог из п. 1.1 .
        1.3.4 Добавление в данном алгоритме происходило по жадному алгоритму. Если у нас оставалось 2 занятия, то мы добавляем к п. 1.3.2 1 занятие в понедельник и во вторник. То есть жадный - добавляем уроки с понедельника.
        
2) Алгоритм построения эталлнного расписания
    2.1 Процесс сборки является итерационным. Итерация выглядит так: собираем расписание для класса -> собираем день -> собираем неделю.
    2.2 На этапе сборки расписания для класса учитываем ограничения:
        2.2.1 - Чтобы более 2 уроков не было в одном дне у одного класса
        2.2.2 - Чтобы учителя не вели урок одновременно в разных классах
        2.2.3 - Кол-во уроков в неделю должно совпадать с кол-вом заданным изначально.
    2.3 Сборка алгоритма начинается с минимального класса - с 5.
        
    

In [4]:
import math

def get_params():

    """ 
    Собрем все возможные параметы.
    """
    
    MAX_LESSONS_IN_DAY = 8
    BASIC_LVL_LESSONS = {
         'Алгебра' : 2,
         'Биология' : 1.5,
         'География' : 1.5,
         'Геометрия' : 2,
         'ИЗО' : 1,
         'Информатика' : 1.5,
         'История' : 1.5,
         'Литература' : 1.5,
         'Математика' : 2,
         'Музыка' : 1,
         'Немецкий язык' : 1.5,
         'ОБЖ' : 1,
         'Общестовазнание' : 1.5,
         'Русский' : 2,
         'Труд' : 1,
         'Физика' : 2,
         'Физкультура' : 1,
         'Французский язык' : 1.5,
         'Химия' : 2         
    }
    
    BASIC_COUNT_STUDY_DAYS = [6 if i == 8 else 5 for i in range(len(COURSES))]
    CLASSES_STANDART_SETTINGS = []
    for clas, basic_count_days in zip(COURSES, BASIC_COUNT_STUDY_DAYS):
        
        
        all_lessons = sum([i.count_lessons for i in clas.values()]) # общее кол-во уроков в неделю
        standart_lessons = math.floor(all_lessons / basic_count_days) # кол-во / 5 или 6
        add_lessons = all_lessons - standart_lessons * basic_count_days
        
        # Жадный алгоритм - добавлять ближе к понедельнику. Есть варианты рандом или 
        count_lessons = [ standart_lessons + 1 if i <= add_lessons else standart_lessons for i in range(1, basic_count_days + 1) ]
        
        count_lvl_lessons = [ BASIC_LVL_LESSONS[i.ItemName[0]] * i.count_lessons for i in clas.values() ]
        avg_lvl = sum(count_lvl_lessons) / basic_count_days
            
#       максимальная сложность в день для класса
#       считаем в каждом классе в разрезе сложности кол-во занятий. 
#       Умножаем кол-во на сложность, суммируем и делим на кол-во дней в базовом варианте. 
#       Округляем до целого (0.5 вврех). Диапозон возможности забития урока = средняя сложнсть - 1 < средняя сложность < средняя сложность + 

        count_lessons.append(avg_lvl)
        CLASSES_STANDART_SETTINGS.append(count_lessons)
    
        print(f'Посчитаем кол-во уроков в будни для класса : {count_lessons} \n')

    
    COUNT_LESSONS_WEEK = []
    for day in range(5):
        COUNT_LESSONS_WEEK.append(sum([i[day] for i in CLASSES_STANDART_SETTINGS]))
    
    return BASIC_LVL_LESSONS, CLASSES_STANDART_SETTINGS, COUNT_LESSONS_WEEK, MAX_LESSONS_IN_DAY
    
    
def get_timetable() -> list:
    """ 
        Начинаем собирать школьное расписание
        Итерируемся пока не закончатся все уроки в каждом дне. Итерация происходит по дням.
        Расписание - матрица 6 на 8
        У каждого класса зафиксирован свой класс
    """
    
    BASIC_LVL_LESSONS, CLASSES_STANDART_SETTINGS, COUNT_LESSONS_WEEK, MAX_LESSONS_IN_DAY = get_params()
    
    RESULT_TIMETABLE = []#[[[i / i for i in range(1,MAX_LESSONS_IN_DAY + 1)] for _ in range(len(COURSES))] for _ in range(len(COUNT_LESSONS_WEEK) + 1)]
    for idx_day, count_lessons_day in tqdm(enumerate(COUNT_LESSONS_WEEK)):    
        timetable_day = get_timetable_day(idx_day, count_lessons_day, RESULT_TIMETABLE, CLASSES_STANDART_SETTINGS, BASIC_LVL_LESSONS, COUNT_LESSONS_WEEK)
              
    return RESULT_TIMETABLE

    
def get_timetable_day( idx_day : int, 
                       count_lessons_day : int, 
                       RESULT_TIMETABLE : list, 
                       CLASSES_STANDART_SETTINGS : list,
                       BASIC_LVL_LESSONS : dict,
                       COUNT_LESSONS_WEEK : list
                      ) -> list:

    """
    Итерируемся по классам. Проверяем, все ли кол-во дней забито.
    """

    count_lessons = 0
    for idx_clas in range(len(CLASSES_STANDART_SETTINGS)):
        timetable_class = insert_courses(idx_clas, idx_day, RESULT_TIMETABLE, CLASSES_STANDART_SETTINGS, BASIC_LVL_LESSONS, COUNT_LESSONS_WEEK)
        count_lessons += sum([0 if i == 0 else 1 for i in timetable_class])
        
        #  Вставляем в расписание новый класс
        if len(RESULT_TIMETABLE) == 0:
            RESULT_TIMETABLE.append([timetable_class])
        elif idx_clas == 0:
            RESULT_TIMETABLE.append([timetable_class])
        else:
            RESULT_TIMETABLE[idx_day].append(timetable_class)
    
    assert count_lessons == count_lessons_day, 'кол-во уроков заданных в базе не совпадает с количеством добавленных уроков!'
    
    return RESULT_TIMETABLE[idx_day]
    

def insert_courses(idx_clas : int,
                   idx_day : int,
                   RESULT_TIMETABLE : list, 
                   CLASSES_STANDART_SETTINGS : list,
                   BASIC_LVL_LESSONS : dict,
                   COUNT_LESSONS_WEEK : list
                  ) -> list:
    """
    Заходим в класс. Проставляем расписание класса за день.
    Рандомно ставим первый урок. После идет проверка на ограничение : Если урок уже был 2 раза проставлен, больше не    может учавствовать в этом дне.
    
    Нагрузка должна быть больше наименьшего порога. Если порог не достигнут или наоборот - превыше, то заново пересчитываем.
    Ограничения, которые поступают сюда, должны учитывать прошлые расчеты. То есть для каждого класса фиксировать использованные уроки.
    """
    
    count_lesson = CLASSES_STANDART_SETTINGS[idx_clas][idx_day]
    flag_constraint = False
    
    while True:
        class_course = []
        for lesson in range(0, 8):
            if lesson <= count_lesson:
                if idx_day == 0 and idx_clas == 0 and lesson == 0:
                    class_course.append(random.choice(list(COURSES[idx_clas].keys())))
                else:
                    access_courses = get_access_courses(lesson, idx_day, idx_clas, RESULT_TIMETABLE, COURSES[idx_clas], class_course)
                    if len(access_courses) == 0: 
                        class_course.append(0)
                    elif len(class_course) == count_lesson:
                        class_course.append(0)
                    else:
                        class_course.append(random.choice(access_courses))   
            else:
                class_course.append(0)
        
        flag_constraint = constrains(idx_clas, class_course, CLASSES_STANDART_SETTINGS, BASIC_LVL_LESSONS)
        if flag_constraint:
            return class_course

def get_access_courses(idx_lesson : int, idx_day : int, idx_clas : int, 
                       RESULT_TIMETABLE : list, courses : dict, class_course : list) -> list :
    """ 
    Формируем доступные курсы на конкретный урок в этот день.
    1) Проверяем доступных учителей в ячейке урока
    2) Убираем курсы, которые уже были 2 раза в этом дне
    3) Убираем курсы, которые уже недоступны.
    
    *3) Убираем курсы, при добавлении которых стоимость дня будет превышена - добавим потом, если будет плохо работать
    """
    
    access_courses = list(courses.keys())
    
    # уберем курсы, которые 2 раза были в этом дне
    for course in set(class_course):
        if class_course.count(course) >= 2:
            access_courses.remove(course)
            
    # Убираем курсы, которые уже недоступны
    used_course = [course[idx_lesson] for day in RESULT_TIMETABLE for course in day] + class_course
    for course in set(used_course):
        if course == 0 or course not in list(COURSES[idx_clas].keys()):
            continue
        if used_course.count(course) == courses[course].count_lessons:
            try:
                access_courses.remove(course)
            except:
                continue
    
    # Проверяем доступных учителей в ячейке урока
    if idx_clas == 0:
        pass
    else:                
        courses_in_lesson = [value[idx_lesson] for idx, value in enumerate(RESULT_TIMETABLE[idx_day]) if idx != idx_clas]
        interseption_teachers = set([i.TeacherFIO for i in COURSES[idx_clas].values()]) & set([COURSES[idx][course].TeacherFIO for idx, course in enumerate(courses_in_lesson) if course != 0])

        if len(interseption_teachers) > 0:
            for course_key, course_value in COURSES[idx_clas].items():
                if course_value.TeacherFIO in interseption_teachers:
                    try:
                         access_courses.remove(course_key)
                    except:
                        continue
    
    return access_courses


def constrains(idx_clas :int, class_course : list, CLASSES_STANDART_SETTINGS : list, BASIC_LVL_LESSONS : dict) -> bool:
    """
    Проверяем стоимость занятий класса на день.
    """
    
    copy_class_course = class_course.copy()
    while True:
        try:
            copy_class_course.remove(0)
        except:
            break
            
    cost = sum([BASIC_LVL_LESSONS[ COURSES[idx_clas][course].ItemName[0] ] for course in copy_class_course])       
    avg_cost = round(CLASSES_STANDART_SETTINGS[idx_clas][-1])
    if avg_cost - 1 <= cost <= avg_cost + 1:
        return True
    else:
        return False
    

In [9]:
timetable = get_timetable()
timetable

5it [00:00, 293.27it/s]

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 



[[[12, 8, 10, 3, 2, 2, 0, 0],
  [4, 2, 3, 12, 3, 8, 0, 0],
  [2, 12, 13, 3, 10, 6, 5, 0],
  [1, 2, 3, 13, 14, 4, 10, 0],
  [1, 4, 10, 17, 3, 12, 2, 0],
  [7, 15, 3, 11, 2, 4, 3, 0],
  [4, 12, 5, 4, 15, 15, 16, 0],
  [3, 1, 10, 3, 10, 11, 12, 14]],
 [[9, 6, 9, 7, 12, 2, 0, 0],
  [10, 2, 10, 9, 9, 5, 0, 0],
  [13, 5, 4, 2, 12, 14, 6, 0],
  [2, 8, 12, 2, 5, 6, 13, 0],
  [6, 16, 2, 16, 2, 17, 6, 0],
  [1, 17, 3, 3, 11, 11, 14, 0],
  [10, 13, 13, 5, 4, 3, 8, 0],
  [14, 15, 7, 9, 3, 3, 2, 0]],
 [[3, 2, 11, 1, 9, 9, 0, 0],
  [10, 9, 2, 3, 11, 11, 0, 0],
  [12, 2, 8, 6, 1, 10, 11, 0],
  [1, 11, 6, 3, 7, 2, 7, 0],
  [17, 2, 4, 14, 16, 1, 1, 0],
  [15, 15, 12, 13, 12, 8, 3, 0],
  [15, 8, 16, 4, 3, 3, 6, 0],
  [9, 14, 13, 8, 7, 12, 3, 0]],
 [[1, 1, 9, 3, 6, 0, 0, 0],
  [12, 9, 1, 11, 1, 0, 0, 0],
  [6, 9, 14, 8, 3, 1, 0, 0],
  [2, 8, 11, 1, 8, 2, 0, 0],
  [2, 2, 9, 7, 12, 12, 10, 0],
  [13, 15, 16, 12, 9, 7, 3, 0],
  [3, 9, 12, 9, 14, 11, 11, 0],
  [7, 3, 4, 1, 11, 15, 5, 0]],
 [[2, 2, 9, 4, 12, 

In [6]:
def get_grah_timetable(timetable : list) -> None:
    
    import pandas as pd
    
    result_df = pd.DataFrame()
    for day in timetable:        
        df_day = pd.DataFrame()
        for idx, clas in enumerate(day):
            temp_df = pd.DataFrame([COURSES[idx][values].ItemName[0] if values != 0 else 0 for values in clas], columns = [f'{idx}'])
            df_day = pd.concat([df_day, temp_df], axis = 1)

        result_df = pd.concat([result_df, df_day])  

    return result_df

def get_grah_teachertimetable(timetable : list) -> None:
    
    import pandas as pd
    
    result_df = pd.DataFrame()
    for day in timetable:        
        df_day = pd.DataFrame()
        for idx, clas in enumerate(day):
            temp_df = pd.DataFrame([COURSES[idx][values].TeacherFIO if values != 0 else 0 for values in clas], columns = [f'{idx}'])
            df_day = pd.concat([df_day, temp_df], axis = 1)

        result_df = pd.concat([result_df, df_day])  

    return result_df

df = get_grah_timetable(timetable)
df

,0,1,2,3,4,5,6,7
0,Математика,Труд,Немецкий язык,Музыка,Русский,Общестовазнание,Биология,Русский
1,ИЗО,Биология,Общестовазнание,Математика,География,Физика,Информатика,Литература
2,Математика,География,ИЗО,Труд,География,Литература,Общестовазнание,ОБЖ
3,История,Музыка,Математика,Литература,История,ОБЖ,Физкультура,Алгебра
4,Физкультура,Немецкий язык,Литература,География,Труд,Французский язык,Алгебра,История
5,Труд,Математика,Физкультура,Общестовазнание,Труд,Литература,Химия,Общестовазнание
6,0,0,Биология,Математика,Информатика,Физкультура,Литература,Общестовазнание
7,0,0,0,0,0,0,0,Физкультура
0,Информатика,Математика,Общестовазнание,Физкультура,Литература,Французский язык,Химия,Геометрия
1,Музыка,Математика,Труд,История,Общестовазнание,Русский,Немецкий язык,Физкультура


In [7]:
get_grah_teachertimetable(timetable)

,0,1,2,3,4,5,6,7
0,Митрохина Альбина Николаевна,Тукова Елена Николаевна,Самарина Валентина Ивановна,Похвалий Екатерина Михайловна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Суханова Анна Львовна,Соловьева Розалия Андреевна
1,Меликаева Налия Кирилловна,Суханова Анна Львовна,Самарина Валентина Ивановна,Митрохина Альбина Николаевна,Сидорова Софья Александровна,Захарова Ирина Ивановна,Селезнева Светлана Валерьевна,Соловьева Розалия Андреевна
2,Митрохина Альбина Николаевна,Ершова Светлана Николаевна,Меликаева Налия Кирилловна,Просвирин Сергей Дмитриевич,Сидорова Софья Александровна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Тукова Елена Николаевна
3,Просвирин Сергей Дмитриевич,Похвалий Екатерина Михайловна,Митрохина Альбина Николаевна,Соловьева Розалия Андреевна,Селезнева Светлана Валерьевна,Тукова Елена Николаевна,Ершова Светлана Николаевна,Захарова Ирина Ивановна
4,Ершова Светлана Николаевна,Самарина Валентина Ивановна,Сухарева Оксана Николаевна,Сидорова Софья Александровна,Просвирин Сергей Дмитриевич,Чуркина Наталья Аркадьевна,Захарова Ирина Ивановна,Селезнева Светлана Валерьевна
5,Тукова Елена Николаевна,Митрохина Альбина Николаевна,Ершова Светлана Николаевна,Самарина Валентина Ивановна,Просвирин Сергей Дмитриевич,Сорокина Анастасия Александровна,Суханова Анна Львовна,Чуркина Наталья Аркадьевна
6,0,0,Суханова Анна Львовна,Митрохина Альбина Николаевна,Селезнева Светлана Валерьевна,Ершова Светлана Николаевна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна
7,0,0,0,0,0,0,0,Ершова Светлана Николаевна
0,Селезнева Светлана Валерьевна,Митрохина Альбина Николаевна,Самарина Валентина Ивановна,Ершова Светлана Николаевна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Суханова Анна Львовна,Захарова Ирина Ивановна
1,Похвалий Екатерина Михайловна,Митрохина Альбина Николаевна,Тукова Елена Николаевна,Просвирин Сергей Дмитриевич,Самарина Валентина Ивановна,Сорокина Анастасия Александровна,Чуркина Наталья Аркадьевна,Ершова Светлана Николаевна


# Модули генетического алгоритма 

### Создание индивидума 

In [286]:
def get_timetable_structure_for_ga(timetable : list) -> list:
    import pandas as pd
    
    result_df = pd.DataFrame()
    for day in timetable:        
        df_day = pd.DataFrame()
        for idx, clas in enumerate(day):
            temp_df = pd.DataFrame(clas, columns = [f'{idx}'])
            df_day = pd.concat([df_day, temp_df], axis = 1)

        result_df = pd.concat([result_df, df_day])  

    return [j for i in result_df.values.tolist() for j in i]

timetable = get_timetable()
get_timetable_structure_for_ga(timetable)

5it [00:00, 362.08it/s]

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 



[9,
 1,
 8,
 4,
 2,
 6,
 10,
 14,
 3,
 1,
 11,
 14,
 6,
 8,
 7,
 1,
 5,
 10,
 12,
 9,
 15,
 8,
 12,
 5,
 11,
 5,
 10,
 8,
 7,
 3,
 17,
 1,
 6,
 7,
 8,
 3,
 2,
 15,
 10,
 6,
 10,
 9,
 5,
 8,
 13,
 2,
 2,
 11,
 0,
 0,
 12,
 3,
 9,
 4,
 1,
 10,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 15,
 12,
 2,
 13,
 7,
 1,
 1,
 9,
 4,
 8,
 12,
 4,
 11,
 10,
 17,
 1,
 4,
 12,
 11,
 7,
 1,
 4,
 6,
 10,
 2,
 3,
 6,
 3,
 11,
 11,
 15,
 12,
 1,
 1,
 11,
 9,
 8,
 17,
 13,
 7,
 8,
 9,
 2,
 8,
 14,
 13,
 4,
 2,
 8,
 0,
 0,
 7,
 6,
 15,
 12,
 2,
 6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 4,
 9,
 1,
 17,
 15,
 15,
 3,
 3,
 7,
 2,
 11,
 9,
 2,
 13,
 12,
 11,
 2,
 10,
 6,
 1,
 5,
 4,
 7,
 9,
 8,
 2,
 4,
 17,
 12,
 3,
 1,
 2,
 8,
 10,
 1,
 16,
 12,
 14,
 5,
 8,
 6,
 12,
 9,
 1,
 2,
 16,
 6,
 0,
 0,
 5,
 13,
 3,
 16,
 16,
 9,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 6,
 8,
 12,
 15,
 3,
 9,
 3,
 9,
 2,
 1,
 12,
 2,
 15,
 10,
 3,
 10,
 1,
 6,
 3,
 6,
 13,
 15,
 12,
 1,
 2,
 8,
 3,
 7,
 16,
 12,
 8,
 8,
 6,
 1,
 10,
 13,
 12,
 17,
 1,

In [287]:
def create_individum() -> list:
    """
    Создание индивидума
    """
    individum = []
    for _ in CLASS_TABLE:
        for i in range(len(CLASSROOMS)):
            individum.append(np.random.randint(0, max(COURSES[i].keys())))
    return individum  

def create_individum() -> list:
    """
    Создание индивидума под новую логику.
    """
    timetable = get_timetable()
    individum = get_timetable_structure_for_ga(timetable)
    return individum 

### Создание популяции 

In [288]:
def create_population(SIZE_POPULATION : int) -> list:
    """
    # Создадим популяцию индивидумов
    """
    return [create_individum() for _ in range(SIZE_POPULATION)]

### Функция, которую будем минимизировать 

In [289]:
def cost(individum : list) -> int:
    """
    Определим фнкцию стоимости индивидума, которую в будущем будем минимизировать
    """
    value = hardConstraintsCountLessons(individum) + \
            hardConstraintsTeachers(individum) 
    return value

#### Модули функции минимизации 

In [290]:
def hardConstraintsCountLessons(individum : list) -> int:
    """
    # 1. hardConstraintsCountLessons - функция дает штраф, если мы поставили больше или меньше занятий по курсу
    """
    count_violation = []
    start_step = 0
    step = len(CLASSROOMS)
    
    a5, b5, a6, b6, a7, b7, a8, a9 = [], [], [], [], [], [], [], []
    
    # Курсы которые содержатся в индивидуме
    while start_step < len(individum):
        
        tmp_individum = individum[start_step: start_step + step]
        for key, lst in enumerate([a5, b5, a6, b6, a7, b7, a8, a9]):
            lst.append(tmp_individum[key]) 
        start_step += step
        
    for key, curs in enumerate(COURSES):
        if key == 0:
            class_ = a5
        if key == 1:
            class_ = b5
        if key == 2:
            class_ = a6
        if key == 3:
            class_ = b6
        if key == 4:
            class_ = a7
        if key == 5:
            class_ = b7
        if key == 6:
            class_ = a8
        if key == 7:
            class_ = a8
            
        count_lesson = [items.count_lessons for key, items in curs.items()] 
        for i in range(max(curs.keys()) + 1):
            count_violation.append(abs(count_lesson[i] - class_.count(i)))   
    return sum(count_violation)

def hardConstraintsTeachers(individum : list) -> int:
    """
    # 2. hardConstraintsTeachers - если преподаватель стоит в один урок на двух разных курсах
    """
    count_violation = 0
    start_step = 0
    step = len(CLASSROOMS)
    
    a5, b5, a6, b6, a7, b7, a8, a9 = [], [], [], [], [], [], [], []
    
    # Курсы которые содержатся в индивидуме
    while start_step < len(individum):
        
        tmp_individum = individum[start_step: start_step + step]
        start_step += step
        
        for key, curs in enumerate(COURSES):
            tmp_individum[key] = curs[tmp_individum[key]].TeacherFIO
            
        count_violation += sum([tmp_individum.count(teacher) for teacher in set(tmp_individum)])
        
    return count_violation

### Функция отбора индивидума турнирным методом 

In [291]:
def select(population : list, COSTS : list, size_tournament : int) -> list:
    """
    # Функция, которая возвращает наилучшего индивидума - турнирный метод
    """
    
    k = len(population) / size_tournament
    best = []
    for _ in range(int(k)):        
        keys_lst = [i for i in range(len(COSTS))]
        idx_list = [random.choice(keys_lst) for _ in range(size_tournament)]       
        best.append(population[sorted([ [idx, COSTS[idx]] for idx in idx_list ], key = lambda x: x[1])[0][0]])
        
    best_costs = sorted([[ind, value] for ind, value in zip(best, [cost(ind) for ind in best])], key = lambda x: x[1])
    return [ind[0] for ind in best_costs] 

### Функция скрещивания индивидумов

In [292]:
def mate(population : list, best_population : list, P_CROSSOVER : float, SIZE_POPULATION :int, P_CROSSOVER_FIRST_PARENT : float) -> list:
    """
    # Функция, которая скрещивает популяцию
    """
    
    new_population = []
    while len(new_population) < SIZE_POPULATION:
        random.shuffle(population)
        for parent1, parent2 in zip(best_population, population):
            if random.random() < P_CROSSOVER:
# Равномерное скрещивание
#                 new_population.append([np.random.choice([par1, par2], 
#                                                         p = [P_CROSSOVER_FIRST_PARENT, 1 - P_CROSSOVER_FIRST_PARENT])\
#                                        for par1, par2 in zip(parent1, parent2)])
                new_population.append([random.choice([par1, par2]) for par1, par2 in zip(parent1, parent2)])
                new_population.append([par2 if add_par == par1 else par1 for add_par, par1, par2 in zip(new_population[-1], parent1, parent2)])
# Одноточечное скрещивание
#                 point = random.randint(0, len(parent1))
#                 new_population.append(parent1[:point] + parent2[point:])
#                 new_population.append(parent2[:point] + parent1[point:])
# Двуточечное скрещивание
#                 point_one = random.randint(0, len(parent1))
#                 point_two = random.randint(point_one, len(parent1))
#                 new_population.append(parent1[point_one:point_two] + parent2[:point_one] + parent2[point_two:])
#                 new_population.append(parent2[point_one:point_two] + parent1[:point_one] + parent1[point_two:])
            else:
                new_population.append(parent1)
                new_population.append(parent2)
        
    return new_population

### Мутация 

In [293]:
def mutate(population : list, P_MUTATE : float) -> list:
    """
    # Функция, которая проводит мутацию с индивидумами - Мутация обменом
    """
    new_population = []
    for mutant in population:
        if random.random() < P_MUTATE:
            for _ in range(2):
                mutant[random.randint(0, len(mutant) - 1)] = 0
        new_population.append(mutant)
    return new_population   

# Генетический алгоритм 

In [296]:
def main() -> list:
    """
    Модули генетического алгоритма, соединенные воедино.
    """
    
    # ****************************************************
    # Константы
    
    np.random.seed(56)  # Чтобы рандом не начинался с нуля 
    random.seed(56)     # Чтобы рандом не начинался с нуля 
    
    SIZE_POPULATION = 25   # Размер популяции
    MAX_GENERATIONS = 400   # Максимальное количество итераций
    P_CROSSOVER_FIRST_PARENT = 0.7 # Вероятность наследования от первого родителя
    
    P_CROSSOVER = 1  # Вероятность скрещивания
    P_MUTATION = 0.5 # Вероятность мутации
    
    # Константы конец
    # ****************************************************
    
    POPULATION = create_population(SIZE_POPULATION)
    
    min_costs = []
    avg_costs = []
    
    for i in tqdm(range(MAX_GENERATIONS)):
        
        COSTS = [cost(ind) for ind in POPULATION]
        NEW_POPULATION = select(POPULATION, COSTS, 4)
        POPULATION = mate(POPULATION, NEW_POPULATION, P_CROSSOVER, SIZE_POPULATION, P_CROSSOVER_FIRST_PARENT)
        POPULATION = mutate(POPULATION[:SIZE_POPULATION], P_MUTATION)
        
        min_costs.append(min(COSTS))
        avg_costs.append(np.mean(COSTS))
        
    
    plt.plot(min_costs, color='red')
    plt.plot(avg_costs, color='green')
    plt.xlabel('Поколение')
    plt.ylabel('Макс/средняя приспособленность')
    plt.title('Зависимость максимальной и средней приспособленности от поколения') 
    plt.show() 
    
    return POPULATION   

In [297]:
d = main()

5it [00:00, 327.22it/s]
5it [00:00, 286.72it/s]
5it [00:00, 433.43it/s]
5it [00:00, 398.69it/s]
5it [00:00, 378.04it/s]

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для кл


5it [00:00, 444.45it/s]
5it [00:00, 293.88it/s]
5it [00:00, 275.77it/s]
5it [00:00, 267.68it/s]
5it [00:00, 339.05it/s]
5it [00:00, 369.14it/s]



Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для 

5it [00:00, 352.73it/s]
5it [00:00, 426.51it/s]
5it [00:00, 318.15it/s]
5it [00:00, 384.86it/s]
5it [00:00, 370.40it/s]
5it [00:00, 378.67it/s]
0it [00:00, ?it/s]

...

5it [00:00, 408.80it/s]
5it [00:00, 347.20it/s]
5it [00:00, 341.11it/s]
5it [00:00, 308.89it/s]
5it [00:00, 309.98it/s]
5it [00:00, 372.74it/s]
0it [00:00, ?it/s]

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для кл

5it [00:00, 347.03it/s]
5it [00:00, 363.72it/s]
  0%|          | 0/400 [00:00<?, ?it/s]


Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [6, 6, 6, 5, 5, 8.9] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 6, 6, 10.4] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 6, 10.6] 

Посчитаем кол-во уроков в будни для класса : [7, 7, 7, 7, 7, 11.3] 

Посчитаем кол-во уроков в будни для класса : [8, 7, 7, 7, 7, 12.0] 



IndexError: list index out of range

# Отображение расписания 

In [24]:
def get_timetabling(individum :list) -> pd.DataFrame:
    """
    Выводим итоговое расписание в виде датафрейма
    """
    
    step = len(CLASSROOMS)
    start_step = 0
    cols = ['day','lesson','a5','b5','a6','b6','a7','b7','a8','a9']
    df = pd.DataFrame(columns = cols)
    day = 1
    lesson = 1

    while start_step < len(individum):
        lst = []
        lst.append(day)
        lst.append(lesson)
        for key, item in enumerate(individum[start_step: start_step + step]):
               lst.append(COURSES[key][item].ItemName[0])
        
        df = pd.concat([df, pd.DataFrame([lst], columns = cols)])
        
        if lesson % 8 == 0:
            lesson = 1
            day += 1
        else:            
            lesson += 1
        start_step += step
        
    return df 

In [25]:
c = [cost(ind) for ind in d]
m = 10000
for ind, cost_ in zip(d, c):
    if cost_ < m:
        best_ind = ind
        
get_timetabling(best_ind)

,day,lesson,a5,b5,a6,b6,a7,b7,a8,a9
0,1,1,Русский,Н,Немецкий язык,Труд,Французский язык,Русский,Н,Н
0,1,2,Н,Н,Н,Французский язык,Н,Труд,География,Н
0,1,3,Н,Литература,Физкультура,Математика,Н,Алгебра,Физкультура,Н
0,1,4,Русский,Н,Немецкий язык,Литература,Физкультура,Алгебра,География,Н
0,1,5,История,Н,История,Математика,Музыка,Н,Общестовазнание,Н
0,1,6,Н,ИЗО,Н,Н,Н,Н,Музыка,Н
0,1,7,Литература,Н,Н,Русский,История,Информатика,Информатика,Н
0,1,8,Н,Русский,Н,Н,Французский язык,Литература,Алгебра,Н
0,2,1,Н,Математика,География,Русский,Алгебра,Литература,Физика,Н
0,2,2,Н,Н,Французский язык,Музыка,История,Музыка,История,Н
